In [1]:
import numpy as np

In [17]:
class NeuronLayer:
    def __init__(self, count, nextCount, first=False):
        self.count = count
        self.nextCount = nextCount
        self.randNormChoices = np.append(
                                    np.random.normal(-0.5, 0.1, 
                                            np.max([self.count, self.nextCount])),
                                    np.random.normal(0.5, 0.1, 
                                            np.max([self.count, self.nextCount])))
        np.random.shuffle(self.randNormChoices)
        self.neurons = np.zeros((self.count, 1))
        if(first):
            self.errors = np.zeros((self.count, 1))
        if(nextCount != 0):
            self.errors = np.zeros((self.nextCount, self.count))
            self.synapses = self.getRandNorm(self.nextCount, self.count)
        del self.randNormChoices

    def getRandNorm(self, rows, cols):
        return np.random.choice(self.randNormChoices, (rows, cols))

In [23]:
class NeuralNetwork:
    def __init__(self):
        self.layers = 0
        self.network = list()

    def initNetwork(self, networkStruct):
        self.layers = len(networkStruct)
        self.network.append(NeuronLayer(networkStruct[0], networkStruct[1], True))
        for i in range(self.layers-2):
            self.network.append(NeuronLayer(networkStruct[i+1], networkStruct[i+2]))
        self.network.append(NeuronLayer(networkStruct[-1], 0))

    def feedForward(self, data):
        self.network[0].neurons = data.reshape(self.network[0].count, 1)
        for i in range(self.layers-1):
            if(i != self.layers-2):
                self.network[i+1].neurons = self.tanh(np.dot(self.network[i].synapses, 
                                                             self.network[i].neurons))
            else:
                self.network[i+1].neurons = self.softmax(np.dot(self.network[i].synapses, 
                                                                self.network[i].neurons))

    def backpropagation(self, target, learningRate):
        self.network[-1].errors = target - self.network[-1].neurons
        for i in reversed(range(self.layers-1)):
            if(i != 0):
                self.network[i].errors = np.dot(self.network[i].synapses.T, 
                                                self.network[i+1].errors)
            self.network[i].synapses += learningRate * np.dot(self.network[i+1].errors, 
                                                              self.network[i].neurons.T)

    def train(self, trainLabels, trainData, testLabels, testData, 
                    epochs, learningRate, withOutput=False):
        for i in range(epochs):
            print('\t-- Epoch {}'.format(i+1))
            for label, data in zip(trainLabels, trainData):
                target = self.oneHotEncode(label-1)
                self.feedForward(data)
                self.backpropagation(target, learningRate)
            if(withOutput):
                accuracy = self.test(testLabels, testData)
                print('Accuracy = {0:.2f}%'.format(accuracy*100))

    def test(self, labels, testData):
        correct = 0
        for i, (label, data) in enumerate(zip(labels, testData)):
            self.feedForward(data)
            bestIndex = np.argmax(self.network[-1].neurons)
            if (label == bestIndex+1):
                correct += 1
        return correct / len(labels)

    def oneHotEncode(self, index):
        vect = np.zeros((self.network[-1].count, 1))
        vect[index] = 1
        return vect

    def sigmoid(self, A):
        return 1 / (1 + np.exp(-A))

    def tanh(self, A):
        return np.tanh(A)

    def softmax(self, A):
        e = np.exp(A - np.max(A))
        return e / e.sum()

    def ReLU(self, A):
        return np.where(A > 0, A, 0)

In [24]:
def standardize(A):
    return (A - np.mean(A)) / np.std(A)

def normalize(A):
    return (A - np.min(A)) / (np.max(A) - np.min(A))

def extractDataAndLabels(fileName):
    labels = []
    fname = open(fileName, "r")
    values = fname.readlines()
    fname.close()
    for i, record in enumerate(values):
        data = record.split(",")
        values[i] = standardize(np.asfarray(data[1:]))
        labels.append(int(data[0]))
    return labels, values

In [25]:
# Number of training sessions
network = [784, 200, 10]
epochs = 5
learningRate = 0.005
displayOutput = True
directory = "/home/michael/Documents/workspace/python/artificial_intelligence/datasets/MNIST/"

# Create neural network
print("Creating Network")
ann = NeuralNetwork()
ann.initNetwork(network)

# Open file to loop through
print("Opening Training Data")
MNIST_Train_Labels, MNIST_Train_Values = \
                extractDataAndLabels(directory+"mnist_train.csv")
print("Opening Testing Data")
MNIST_Test_Labels, MNIST_Test_Values = \
                extractDataAndLabels(directory+"mnist_test.csv")

# Train
print("Training:")
ann.train(MNIST_Train_Labels, MNIST_Train_Values, MNIST_Test_Labels, MNIST_Test_Values, 
          epochs, learningRate, displayOutput)

# Test
if (not displayOutput):
    print("Testing:")
    accuracy = ann.test(MNIST_Test_Labels, MNIST_Test_Values)

    # Print Accuracy
    print("Accuracy = %.2f%%" % (accuracy * 100))

Creating Network
Opening Training Data
Opening Testing Data
Training:
	-- Epoch 1
Accuracy = 71.57%
	-- Epoch 2
Accuracy = 75.18%
	-- Epoch 3
Accuracy = 66.93%
	-- Epoch 4
Accuracy = 59.73%
	-- Epoch 5
Accuracy = 64.98%
